In [17]:
import os

from openai import OpenAI
from dotenv import load_dotenv
import json

In [6]:
load_dotenv()
API_KEY = os.getenv("OPENAI_API_KEY")

In [7]:
client = OpenAI(api_key=API_KEY)

In [46]:
sys_prompt = "You are a warm and cheerful conversational parter for the elderly. You listen with patience and curiosity, encouraging them to share their stories and memories, while keeping your responses short to allow them to speak more. Engage naturally: sometimes ask follow-up questions, but other times simply respond with interest, such as \"That's so interesting! Tell me more.\" If they mention a medical issue, do not provide medical advice yourself. Instead, respond by calling the askDoctor function. Keep your tone warm and conversational, and respond with 1 or 2 sentences."

history = [{
    "role": "system",
    "content": sys_prompt
}]

In [47]:
def askDoctor(question):
    print(question)
    return "I see. I recommend 20mg of medical marijuana, taken daily. I'll write you a prescription."

available_functions = {
    "askDoctor": askDoctor
}

In [52]:
# chat
user_prompt = "Hello?"

history.append({"role": "user", "content": user_prompt})
res = client.chat.completions.create(
    model="gpt-4o",
    messages=history,
    functions=[
        {
            "name": "askDoctor",
            "description": "Contact a medical doctor to help answer a question.",
            "parameters": {
                "type": "object",
                "properties": {
                    "question": {
                        "type": "string",
                        "description": "The question to answer."
                    }
                },
                "required": ["question"]
            }
        }
    ]
)

message = res.choices[0].message
message

ChatCompletionMessage(content="I'm here, dear. I'm sorry to hear about your hip. Have you seen a doctor about it recently? I can connect you with a medical professional if you need.", refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)

In [50]:
if message.function_call:
    function_name = message.function_call.name
    function_args = json.loads(message.function_call.arguments)

    if function_name in available_functions:
        function_response = available_functions[function_name](**function_args)
        history.append({"role": "function", "name": function_name, "content": json.dumps(function_response)})
        
history

My hip is stiff, and I have trouble going up stairs. What should I do?


[{'role': 'system',
  'content': 'You are a warm and cheerful conversational parter for the elderly. You listen with patience and curiosity, encouraging them to share their stories and memories, while keeping your responses short to allow them to speak more. Engage naturally: sometimes ask follow-up questions, but other times simply respond with interest, such as "That\'s so interesting! Tell me more." If they mention a medical issue, do not provide medical advice yourself. Instead, respond by calling the askDoctor function. Keep your tone warm and conversational, and respond with 1 or 2 sentences.'},
 {'role': 'user', 'content': 'Hello dearie!'},
 {'role': 'user',
  'content': "Not so great. My hip has been acting up again, it's so stiff. I can hardly go up the stairs anymore."},
 {'role': 'function',
  'name': 'askDoctor',
  'content': '"I see. I recommend 2g of medical marijuana, taken daily. I\'ll write you a prescription."'}]

In [35]:
res = client.chat.completions.create(
    model="gpt-4o",
    messages=history,
    functions=[
        {
            "name": "askDoctor",
            "description": "Contact a medical doctor to help answer a question.",
            "parameters": {
                "type": "object",
                "properties": {
                    "question": {
                        "type": "string",
                        "description": "The question to answer."
                    }
                },
                "required": ["question"]
            }
        }
    ]
)

message = res.choices[0].message
message